In [1]:
# Import the registration file so that the environment is registered.
# Adjust the import statement below based on where your registration code is.
import gymnasium_env.gymnasium_env  # This file should contain your register(...) call

%reload_ext autoreload
%autoreload 2

# Verify registration.
from gymnasium.envs.registration import registry
print("Registered environments:")
for env_id in registry.keys():
    print(env_id)

Registered environments:
CartPole-v0
CartPole-v1
MountainCar-v0
MountainCarContinuous-v0
Pendulum-v1
Acrobot-v1
phys2d/CartPole-v0
phys2d/CartPole-v1
phys2d/Pendulum-v0
LunarLander-v3
LunarLanderContinuous-v3
BipedalWalker-v3
BipedalWalkerHardcore-v3
CarRacing-v3
Blackjack-v1
FrozenLake-v1
FrozenLake8x8-v1
CliffWalking-v0
Taxi-v3
tabular/Blackjack-v0
tabular/CliffWalking-v0
Reacher-v2
Reacher-v4
Reacher-v5
Pusher-v2
Pusher-v4
Pusher-v5
InvertedPendulum-v2
InvertedPendulum-v4
InvertedPendulum-v5
InvertedDoublePendulum-v2
InvertedDoublePendulum-v4
InvertedDoublePendulum-v5
HalfCheetah-v2
HalfCheetah-v3
HalfCheetah-v4
HalfCheetah-v5
Hopper-v2
Hopper-v3
Hopper-v4
Hopper-v5
Swimmer-v2
Swimmer-v3
Swimmer-v4
Swimmer-v5
Walker2d-v2
Walker2d-v3
Walker2d-v4
Walker2d-v5
Ant-v2
Ant-v3
Ant-v4
Ant-v5
Humanoid-v2
Humanoid-v3
Humanoid-v4
Humanoid-v5
HumanoidStandup-v2
HumanoidStandup-v4
HumanoidStandup-v5
GymV21Environment-v0
GymV26Environment-v0
GymAdapter-v0


In [2]:
import gymnasium as gym
import torch
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

import cuda_square_simulation as CudaUnderlyingEnv

In [3]:
nb_batches = 100
rows, cols = 50, 50
populations = {
    "red": {"p": 0.2, "mean_v": 1.0, "std_v": 0.2},
    "blue": {"p": 0.2, "mean_v": 1.0, "std_v": 0.2},
    "green": {"p": 0.2, "mean_v": 1.0, "std_v": 0.2},
}

In [4]:
# Parameters for the simulation.
observation_size = 5          # Should match the observation dimensions expected by your env.
clever_pop_id = 1             # The index of the population controlled by the RL agent.

# Create an instance of the underlying environment.
underlying_env = CudaUnderlyingEnv.SquareSimulation(
    nb_batch=nb_batches,
    rows=rows,
    cols=cols,
    populations=populations,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    observation_size=observation_size
)

In [5]:
# Create your custom environment instance.
env = gym.make(
    "GymAdapter-v0",
    efficient_environment=underlying_env,  # Pass your underlying environment.
    observation_size=5,                     # Should match your expected observation dimensions.
    clever_pop_id=1,                        # The index corresponding to the clever population.
    render_mode=None
)

In [6]:
# Stable Baselines 3 expects vectorized environments.
# Here we wrap the custom environment in a DummyVecEnv.
vec_env = DummyVecEnv([lambda: env])

# Create the PPO model using an MLP policy.
# The model will learn to control the clever population in the simulation.
model = PPO("MlpPolicy", vec_env, verbose=1)

# Train the model for a given number of timesteps.
model.learn(total_timesteps=10000)

# Save the trained model to a file.
model.save("ppo_clever_population")
print("Model trained and saved successfully.")

# Optional: Evaluate the trained model.
obs, _ = env.reset()
for step in range(10):
    # Predict the action for the clever population given the current observation.
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {step + 1}: Action taken: {action}, Reward: {reward}")
    if terminated.any():
        break

Using cuda device


c:\Users\sebge\miniconda3\envs\shs-philo\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(
c:\Users\sebge\miniconda3\envs\shs-philo\lib\site-packages\gymnasium\utils\passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


ValueError: could not broadcast input array from shape (50413,3,5,5) into shape (3,5,5)